# D09 Tổng hợp dữ liệu (aggregation): kết hợp `groupby()` và `apply()`

## Mục đích

Tổng hợp các tính năng `groupby()` và `apply()` để tổng hợp số liệu.

## Ví dụ

Chúng ta sẽ thống kê trung bình và độ lệch chuẩn (SD) của chiều cao của đối tượng nghiên cứu theo giới tính. Sau khi đã học về `groupby()`, hi vọng bạn đã cảm thấy dễ dàng trong việc này.

In [1]:
import pandas as pd
import numpy as np

d = pd.read_excel("hrm.xlsx", index_col="id")
d_agg = d.groupby("sex")["height"].agg(["mean", "std"])
d_agg

,mean,std
sex,,
0,1.557678,0.044879
1,1.670381,0.059999


Thông thường khi trình bày bảng kết quả (ví dụ, trong PowerPoint, bài báo), chúng ta không để riêng trung bình và SD, mà sẽ gộp chung chúng lại dưới dạng "mean (SD)", và sẽ làm tròn, ví dụ đến 2 chữ số sau dấu thập phân. Nhìn vào bảng trên, bạn dễ dàng thấy rằng chúng ta có thể sử dụng hàm `apply()` để làm việc này.

In [2]:
d_agg.apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1)

sex
0    1.56 (0.04)
1    1.67 (0.06)
dtype: object

Bạn có thể "chain" các hàm này với nhau để không phải tạo ra biến trung gian `d_agg`. Khi dòng lênh quá dài, bạn có thể xuống dòng và đặt dấu `\` ở cuối dòng để nối các dòng lệnh với nhau.

In [3]:
d.groupby("sex")["height"].agg(["mean", "std"]) \
    .apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1)

sex
0    1.56 (0.04)
1    1.67 (0.06)
dtype: object

## Phát triển từ ví dụ

### Tạo ra nhiều kết quả trong một lần `apply()`

Nếu cùng một lúc chúng ta muốn tạo ra "mean (SD)" và "min-max", bạn sẽ cần chú ý và cẩn thận hơn. Đầu tiên, chúng ta sẽ thực hiện `groupby()`.

In [4]:
d.groupby("sex")["height"].agg(["mean", "std", "min", "max"])

,mean,std,min,max
sex,,,,
0,1.557678,0.044879,1.45,1.70
1,1.670381,0.059999,1.50,1.85


Thay vì viết thẳng nội dung vào hàm lambda khi `apply()`, bạn có thể viết riêng một hàm mới cho dễ quản lí.

In [5]:
def get_stats(mean, sd, min, max):
    return f"{mean:.2f} ({sd:.2f})", f"{min:.2f}-{max:.2f}"

d.groupby("sex")["height"].agg(["mean", "std", "min", "max"]) \
    .apply(lambda x: get_stats(*x), axis=1)

sex
0    (1.56 (0.04), 1.45-1.70)
1    (1.67 (0.06), 1.50-1.85)
dtype: object

Hãy nhớ rằng kết quả trả về của mỗi bản ghi là một danh sách. Chúng ta sẽ chuyển danh sách này thành các cột, và đổi tên cho các cột.

In [6]:
d.groupby("sex")["height"].agg(["mean", "std", "min", "max"]) \
    .apply(lambda x: get_stats(*x), axis=1, result_type="expand") \
    .rename(columns={0: "Mean (SD)", 1: "Min-Max"})

,Mean (SD),Min-Max
sex,,
0,1.56 (0.04),1.45-1.70
1,1.67 (0.06),1.50-1.85


### Tổng hợp trên nhiều biến khác nhau

Ở trên chúng ta mới thống kê trên một biến chiều cao. Hãy thử thống kê đồng thời cho biến chiều cao và cân nặng.

In [7]:
d.groupby("sex")[["height", "weight"]].agg(["mean", "std", "min", "max"])

height                           weight                      
         mean       std   min   max       mean       std   min   max
sex                                                                 
0    1.557678  0.044879  1.45  1.70  52.045662  5.920553  39.0  72.0
1    1.670381  0.059999  1.50  1.85  63.285714  8.388080  46.0  90.0

Bạn sẽ thấy rằng chúng ta đang tạo ra một danh sách các cột có tên đa tầng. Để tiếp tục sử dụng `apply()` bạn sẽ phải chuyển chiều cao và cân nặng thành hàng, để chỉ còn 4 cột "mean" đến "max". Chúng ta sẽ làm việc này trong bài nói về biến đổi cấu trúc dữ liệu.

Một giải pháp khác là tạo ra kết quả tổng hợp dữ liệu cho từng biến, sau đó gộp các kết quả này lại. Cách làm này sẽ thuận lợi trong một số trường hợp và bạn nên biết (và thành thạo) cách làm này.

Đầu tiên, chúng ta sẽ tạo ra một hàm để sinh kết quả tổng hợp cho một biến.

In [8]:
def get_agg(d, groupby, var_to_agg):
    return d.groupby(groupby)[var_to_agg].agg(["mean", "std", "min", "max"]) \
        .apply(lambda x: get_stats(*x), axis=1, result_type="expand") \
        .rename(columns={0: "Mean (SD)", 1: "Min-Max"})

get_agg(d, "sex", "height")

,Mean (SD),Min-Max
sex,,
0,1.56 (0.04),1.45-1.70
1,1.67 (0.06),1.50-1.85


Bạn sẽ nhận thấy rằng chúng ta không có thông tin là biến nào đang được tổng hợp. Hãy thêm một cột tên biến vào kết quả trước khi trả về.

In [9]:
def get_agg(d, groupby, var_to_agg):
    d_agg = d.groupby(groupby)[var_to_agg].agg(["mean", "std", "min", "max"]) \
        .apply(lambda x: get_stats(*x), axis=1, result_type="expand") \
        .rename(columns={0: "Mean (SD)", 1: "Min-Max"})
    d_agg["Variable"] = var_to_agg
    return d_agg

get_agg(d, "sex", "height")

,Mean (SD),Min-Max,Variable
sex,,,
0,1.56 (0.04),1.45-1.70,height
1,1.67 (0.06),1.50-1.85,height


Để lặp lại việc chạy hàm này trên nhiều biến, chúng ta sẽ dùng hàm `map()`. Nếu bạn chưa nhớ hàm này, hãy ôn lại bài [I02](../02_inter/02_lambda.ipynb). Hàm `map()` trả về một iterator của các đối tượng là data frame, và chúng ta sẽ gộp các data frame này lại.

In [10]:
vars_to_agg = ["height", "weight"]
pd.concat(list(map(lambda x: get_agg(d, "sex", x), vars_to_agg)))

,Mean (SD),Min-Max,Variable
sex,,,
0,1.56 (0.04),1.45-1.70,height
1,1.67 (0.06),1.50-1.85,height
0,52.05 (5.92),39.00-72.00,weight
1,63.29 (8.39),46.00-90.00,weight


Một chút xíu điều chỉnh nhỏ nhỏ: Đặt index của data frame này theo biến được tổng hợp, sau đó theo nhóm.

In [11]:
pd.concat(list(map(lambda x: get_agg(d, "sex", x), vars_to_agg))) \
    .reset_index().set_index(["Variable", "sex"])

Mean (SD)      Min-Max
Variable sex                           
height   0     1.56 (0.04)    1.45-1.70
         1     1.67 (0.06)    1.50-1.85
weight   0    52.05 (5.92)  39.00-72.00
         1    63.29 (8.39)  46.00-90.00

### Nhóm nhiều nhóm cùng một lúc

Trong hàm `get_agg()` ở trên, đối số `groupby` không quy định kiểu rõ ràng, nên chúng ta có thể sử dụng tất cả các kiểu dữ liệu mà hàm `groupby()` của data frame chấp nhận. Ví dụ, chúng ta nhóm theo giới và tuổi >45.

In [12]:
d["age"] = d["date_exam"].dt.year - d["yob"]
d["age_gt45"] = d["age"].gt(45).astype(int)

vars_groupby = ["age_gt45", "sex"]
get_agg(d, vars_groupby, "height")

Mean (SD)    Min-Max Variable
age_gt45 sex                                 
0        0    1.57 (0.04)  1.45-1.70   height
         1    1.68 (0.06)  1.50-1.85   height
1        0    1.55 (0.04)  1.45-1.70   height
         1    1.66 (0.06)  1.50-1.80   height

Chúng ta cũng có thể nhóm nhiều nhóm nhưng theo từng nhóm một giống như phần trên.

In [13]:
def get_agg(d, groupby, var_to_agg):
    d_agg = d.groupby(groupby)[var_to_agg].agg(["mean", "std", "min", "max"]) \
        .apply(lambda x: get_stats(*x), axis=1, result_type="expand") \
        .rename(columns={0: "Mean (SD)", 1: "Min-Max"})
    d_agg["Group by"] = groupby
    d_agg["Variable"] = var_to_agg
    return d_agg

pd.concat(list(map(lambda x: get_agg(d, x, "height"), vars_groupby))) \
    .rename_axis(index="Level").reset_index().set_index(["Group by", "Level"])

Mean (SD)    Min-Max Variable
Group by Level                                 
age_gt45 0      1.61 (0.07)  1.45-1.85   height
         1      1.58 (0.07)  1.45-1.80   height
sex      0      1.56 (0.04)  1.45-1.70   height
         1      1.67 (0.06)  1.50-1.85   height

Nếu muốn lặp lại cả danh sách nhóm và biến tổng hợp dữ liệu thì làm thế nào? Bạn sẽ cần tạo ra một danh sách chứa tất cả các cặp "nhóm - biến". Chúng ta có thể sử dụng [generator](../02_inter/03_yield.ipynb) cho việc này. Với generator bạn sẽ không phải tạo ra một danh sách hoặc tuple của tất cả các cặp "nhóm - biến".

In [14]:
vars_all_pairs = ((groupby, var_to_agg) for groupby in vars_groupby for var_to_agg in vars_to_agg)
pd.concat(list(map(lambda x: get_agg(d, *x), vars_all_pairs))) \
    .rename_axis(index="Level").reset_index().set_index(["Variable", "Group by", "Level"])

Mean (SD)      Min-Max
Variable Group by Level                           
height   age_gt45 0       1.61 (0.07)    1.45-1.85
                  1       1.58 (0.07)    1.45-1.80
weight   age_gt45 0      55.90 (9.11)  39.00-90.00
                  1      55.49 (8.14)  40.00-82.00
height   sex      0       1.56 (0.04)    1.45-1.70
                  1       1.67 (0.06)    1.50-1.85
weight   sex      0      52.05 (5.92)  39.00-72.00
                  1      63.29 (8.39)  46.00-90.00

---

[Bài trước](./07_apply.ipynb) - [Danh sách bài](../README.md) - [Bài sau]()